<a href="https://colab.research.google.com/github/Jiayi1827/STAT-430/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
#TODO: indicate whether you want to use Numpy or Pytorch here by modifying variable 'use_pytorch'
# If you want to use Pytorch leave it as default True, otherwise set it to False
use_pytorch = False

In [ ]:
# Problem Naive Bayes
def bayes_MAP(X, y):
    '''
    Arguments:
        X (S x N LongTensor / Numpy ndarray): features of each object, X[i][j] = 0/1
        y (S LongTensor  / Numpy ndarray): label of each object, y[i] = 0/1

    Returns:
        D (2 x N Float Tensor / Numpy ndarray): MAP estimation of P(X_j=1|Y=i)

    '''
    N, d = X.shape
    D = np.zeros((2, d))

    # Estimate P(X_j=1|Y=0)
    for j in range(d):
        D[0, j] = (np.sum((X[:, j] == 1) & (y == 0)) + 1) / (np.sum(y == 0) + 2)  # Beta(1, 1) prior

    # Estimate P(X_j=1|Y=1)
    for j in range(d):
        D[1, j] = (np.sum((X[:, j] == 1) & (y == 1)) + 1) / (np.sum(y == 1) + 2)  # Beta(1, 1) prior

    return D


def bayes_MLE(y):
    '''
    Arguments:
        y (S LongTensor / Numpy ndarray): label of each object

    Returns:
        p (float or scalar Float Tensor / Numpy ndarray): MLE of P(Y=1)

    '''
    return np.mean(y)



def bayes_classify(D, p, X):
    '''
    Arguments:
        D (2 x N Float Tensor / Numpy ndarray): returned value of `bayes_MAP`
        p (float or scalar Float Tensor / Numpy ndarray): returned value of `bayes_MLE`
        X (S x N LongTensor / Numpy ndarray): features of each object for classification, X[i][j] = 0/1

    Returns:
        y (S LongTensor / Numpy ndarray): label of each object for classification, y[i] = 0/1

    '''
    N, d = X.shape
    log_probs_0 = np.sum(X * np.log(D[0, :]) + (1 - X) * np.log(1 - D[0, :]), axis=1) + np.log(1 - p)
    log_probs_1 = np.sum(X * np.log(D[1, :]) + (1 - X) * np.log(1 - D[1, :]), axis=1) + np.log(p)

    # If log_probs_1 > log_probs_0, return 1, else return 0
    return (log_probs_1 > log_probs_0).astype(int)


In [ ]:
# Problem Gaussian Naive Bayes
def gaussian_MAP(X, y):
    '''
    Arguments:
        X (S x N FloatTensor / Numpy ndarray): features of each object
        y (S LongTensor / Numpy ndarray): label of each object, y[i] = 0/1

    Returns:
        mu (2 x N Float Tensor / Numpy ndarray): MAP estimation of mu in N(mu, sigma2)
        sigma2 (2 x N Float Tensor / Numpy ndarray): MAP estimation of mu in N(mu, sigma2)

    '''
    _, N = X.shape
    mu = np.zeros((2, N))
    sigma2 = np.zeros((2, N))

    for c in [0, 1]:
        X_c = X[y == c]
        mu[c, :] = np.mean(X_c, axis=0)
        sigma2[c, :] = np.var(X_c, axis=0)  # variance, not standard deviation

    return mu, sigma2


def gaussian_MLE(y):
    '''
    Arguments:
        y (S LongTensor / Numpy ndarray): label of each object

    Returns:
        p (float or scalar Float Tensor / Numpy ndarray): MLE of P(Y=1)

    '''
    return np.mean(y)

def gaussian_classify(mu, sigma2, p, X):
    '''
    Arguments:
        mu (2 x N Float Tensor / Numpy ndarray): returned value #1 of `gaussian_MAP` (estimation of mean)
        sigma2 (2 x N Float Tensor / Numpy ndarray): returned value #2 of `gaussian_MAP` (square of sigma)
        p (float or scalar Float Tensor / Numpy ndarray): returned value of `bayes_MLE`
        X (S x N LongTensor / Numpy ndarray): features of each object for classification, X[i][j] = 0/1

    Returns:
        y (S LongTensor / Numpy ndarray): label of each object for classification, y[i] = 0/1

    '''
    N, d = X.shape
    log_prob = np.zeros((N, 2))

    for c in [0, 1]:
        log_prob_class = np.sum(-0.5 * np.log(2 * np.pi * sigma2[c, :])
                                - 0.5 * (X - mu[c, :])**2 / sigma2[c, :], axis=1)
        log_prob[:, c] = log_prob_class + np.log(p if c == 1 else (1-p))

    return np.argmax(log_prob, axis=1).astype(np.int)


    pass